In [121]:
from extraction.extract import *
from extraction.extractionvalues import *
from extraction.airportvalues import *
import numpy as np


import pandas as pd


In [103]:
Load = extractData(start=datetime(2019, 3, 1), end=datetime(2019, 3, 31))
print(len(Load))
Load.head()


100%|██████████| 3/3 [01:00<00:00, 20.28s/it]


2398503


,ECTRLID,ADEP,ADEPLat,ADEPLong,ADES,ADESLat,ADESLong,FiledOBT,FiledAT,ActualOBT,ActualAT,ACType,ACOperator,FlightType,ActualDistanceFlown
0,227743250,VOMM,12.99361,80.17694,LFPG,49.00972,2.54778,2019-03-01,2019-03-01 11:08:54,2019-02-28 23:59:19,2019-03-01 11:14:19,A333,JAI,Traditional Scheduled,4518
1,227743251,CYYZ,43.67722,-79.63056,LIMC,45.63000,8.72306,2019-03-01,2019-03-01 07:42:47,2019-03-01 00:28:00,2019-03-01 08:16:42,B763,ACA,Traditional Scheduled,3802
2,227743252,KORD,41.98000,-87.90500,EHAM,52.30806,4.76417,2019-03-01,2019-03-01 07:58:41,2019-03-01 00:00:00,2019-03-01 08:10:34,B763,UAL,Traditional Scheduled,3805
3,227743253,KEWR,40.69667,-74.16833,LSZH,47.45806,8.54806,2019-03-01,2019-03-01 07:14:53,2019-03-01 00:12:00,2019-03-01 07:41:18,B764,UAL,Traditional Scheduled,3553
4,227743254,KJFK,40.64028,-73.77833,EGLL,51.47750,-0.46139,2019-03-01,2019-03-01 06:35:20,2019-03-01 00:20:00,2019-03-01 07:02:22,A332,DAL,Traditional Scheduled,3072


In [106]:
K = filterAirports(Load, ICAOTOP10)
K = calculateDelays(K)
# print(len(K))
K.head()


,ECTRLID,ADEP,ADEPLat,ADEPLong,ADES,ADESLat,ADESLong,FiledOBT,FiledAT,ActualOBT,ActualAT,ACType,ACOperator,FlightType,ActualDistanceFlown,ArrivalDelay,DepartureDelay
400,227743837,UUEE,55.97250,37.41306,EHAM,52.30806,4.76417,2019-03-01 02:30:00,2019-03-01 05:30:27,2019-03-01 03:07:00,2019-03-01 06:10:31,B738,KLM,Traditional Scheduled,1187,40.0,37.0
716,227744326,UUEE,55.97250,37.41306,LEMD,40.47222,-3.56083,2019-03-01 03:50:00,2019-03-01 08:39:59,2019-03-01 04:04:00,2019-03-01 09:03:02,A321,AFL,Traditional Scheduled,1900,23.0,14.0
726,227744338,UUEE,55.97250,37.41306,LFPG,49.00972,2.54778,2019-03-01 03:50:00,2019-03-01 07:17:05,2019-03-01 04:30:00,2019-03-01 08:05:17,A320,AFR,Traditional Scheduled,1355,48.0,40.0
1101,227744804,UUEE,55.97250,37.41306,EDDM,48.35389,11.78611,2019-03-01 04:55:00,2019-03-01 07:50:28,2019-03-01 04:58:00,2019-03-01 07:47:08,A321,AFL,Traditional Scheduled,1067,-4.0,3.0
1168,227744887,LEMD,40.47222,-3.56083,EHAM,52.30806,4.76417,2019-03-01 05:00:00,2019-03-01 07:17:23,2019-03-01 05:44:00,2019-03-01 08:05:54,B738,KLM,Traditional Scheduled,809,48.0,44.0


In [215]:
P = K.drop(
    [
        "ACType",
        "ACOperator",
        "FlightType",
        "ActualDistanceFlown",
        "ECTRLID",
        "ADEPLat",
        "ADEPLong",
        "ADESLat",
        "ADESLong",
        "ActualOBT",
        "ActualAT",
        "ArrivalDelay",
        "DepartureDelay",
    ],
    axis=1,
)
P.sort_values("FiledOBT")
print(P.head(10))
print(P.shape)


      ADEP  ADES            FiledOBT             FiledAT
400   UUEE  EHAM 2019-03-01 02:30:00 2019-03-01 05:30:27
716   UUEE  LEMD 2019-03-01 03:50:00 2019-03-01 08:39:59
726   UUEE  LFPG 2019-03-01 03:50:00 2019-03-01 07:17:05
1101  UUEE  EDDM 2019-03-01 04:55:00 2019-03-01 07:50:28
1168  LEMD  EHAM 2019-03-01 05:00:00 2019-03-01 07:17:23
1173  UUEE  EGLL 2019-03-01 05:00:00 2019-03-01 08:36:02
1223  LEBL  LEMD 2019-03-01 05:00:00 2019-03-01 05:59:46
1233  LEBL  EDDF 2019-03-01 05:00:00 2019-03-01 07:11:01
1246  LEBL  EHAM 2019-03-01 05:00:00 2019-03-01 07:07:39
1457  LEMD  EDDF 2019-03-01 05:15:00 2019-03-01 07:52:49
(65825, 4)


In [214]:
dep = P.groupby([pd.Grouper(key="FiledOBT", freq=f"{15}min"),]).agg(
    {"ADES": "count"}
)  # ["ADES"].value_counts().unstack(fill_value=0)
dep.head(30)
# dep.to_numpy()


ADES
ADEP FiledOBT                 
EDDF 2019-03-01 05:15:00     1
     2019-03-01 05:45:00     3
     2019-03-01 06:00:00     3
     2019-03-01 06:15:00     1
     2019-03-01 06:30:00     1
     2019-03-01 06:45:00     2
     2019-03-01 07:15:00     1
     2019-03-01 07:30:00     1
     2019-03-01 08:00:00     2
     2019-03-01 08:15:00     2
     2019-03-01 08:30:00     1
     2019-03-01 09:00:00     2
     2019-03-01 09:15:00     2
     2019-03-01 09:45:00     2
     2019-03-01 10:00:00     1
     2019-03-01 10:15:00     3
     2019-03-01 10:45:00     1
     2019-03-01 11:00:00     4
     2019-03-01 11:15:00     2
     2019-03-01 11:30:00     1
     2019-03-01 12:15:00     1
     2019-03-01 12:30:00     1
     2019-03-01 12:45:00     1
     2019-03-01 13:00:00     1
     2019-03-01 13:15:00     3
     2019-03-01 13:30:00     1
     2019-03-01 14:15:00     2
     2019-03-01 15:00:00     3
     2019-03-01 15:15:00     1
     2019-03-01 15:30:00     2

In [303]:
arr = P.groupby([ pd.Grouper(key="FiledAT", freq=f"{60}min"),
                "ADES"])["ADEP"].value_counts().unstack(fill_value=0)

mux = pd.MultiIndex.from_product([arr.index.get_level_values("FiledAT").unique(), arr.index.get_level_values("ADES").unique()])
# df = arr.pd.MultiIndex.sortlevel(level = ["FiledAT", "ADES"] , ascending= ICAOTOP10     )                      
df = arr.reindex(mux)
# df.iloc[:]["ADES"]

a  = (df.sort_index(level=0, ascending=True)
        .reindex(sorted(df.columns), axis=1)
        .fillna(0))
a = a.to_numpy()
maximum = np.zeros((10,10))

b = [a[i:i+10] for i in range(0,len(a), 10)]
b = np.array(b)
for i in range(len(a[0])):
        for j in range(len(a[0])):
                maximum[i][j] = max(b[:,i,j])
adj_matrices_demand = np.array([x / maximums_array for x in b])
